In [ ]:
from qiskit import QuantumCircuit, transpile, execute
from qiskit_aer import AerSimulator
import numpy as np
from typing import List, Tuple



1) Create Receiver class
2) Take in the sent bits
3) Write a function to incorporate noise to the sent bits
4) Take the sent bits and measure them with or without noise - basically noise should be optional to incorporate


In [ ]:

def generate_bases(n_bits: int) -> np.ndarray:
    """Generate random measurement bases for Bob: 0 = Z, 1 = X."""
    return np.random.randint(2, size=n_bits)


In [ ]:


def apply_noise(circuits: List[QuantumCircuit], noise_type: str = "none", noise_prob: float = 0.05) -> List[QuantumCircuit]:
    """Apply optional noise to each qubit circuit in the list."""
    noisy_circuits = []

    for qc in circuits:
        noisy_qc = qc.copy()  # make a copy to avoid modifying original

        # Apply noise probabilistically
        if noise_type != "none" and np.random.rand() < noise_prob:
            if noise_type == "bit-flip":
                noisy_qc.x(0)
            elif noise_type == "phase-flip":
                noisy_qc.z(0)
            elif noise_type == "depolarizing":
                gate = np.random.choice(["x", "y", "z"])
                if gate == "x":
                    noisy_qc.x(0)
                elif gate == "y":
                    noisy_qc.y(0)
                else:
                    noisy_qc.z(0)

        noisy_circuits.append(noisy_qc)

    return noisy_circuits


In [ ]:
def measure_qubits(circuits, bob_bases, noise_type: str = "none", noise_prob: float = 0.05):
    backend = AerSimulator()
    measured_bits = []

    for qc, basis in zip(circuits, bob_bases):
        noisy_qc = qc.copy()

        # Apply optional noise
        noisy_qc = apply_noise(noisy_qc, noise_type, noise_prob)

        # Apply Bob's measurement basis
        if basis == 1:  # X-basis
            noisy_qc.h(0)

        # Measure
        noisy_qc.measure(0, 0)
        transpiled_qc = transpile(noisy_qc, backend)
        job = backend.run(transpiled_qc, shots=1, memory=True)
        result = job.result()
        bit = int(result.get_memory()[0])
        measured_bits.append(bit)

    return measured_bits